In [ ]:
# --- Simulation Strategy Backtesting ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Data generation ---

np.random.seed(42)
n_steps = 252
n_simulations = 1
T = 1
dt = T / n_simulations

# --- Price Process Generation ---

mu = 0.06
S0 = 100
sigma = 0.20
dZ = np.random.standard_normal((n_steps, n_simulations))
S = np.zeros((n_steps, n_simulations))

for i in range(n_simulations):
    S[0, i] = S0
    for t in range(1, n_steps):
        S[t, i] = S[t - 1, i] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * dZ[t, i])

S_df = pd.DataFrame(S, columns = ['S_t'])

# --- Log Returns Computation ---

S_df['r_t'] = np.log(S_df['S_t']).diff(1)
S_df = S_df.ffill().fillna(0)
S_df

# --- 21-day rolling window RV ---

window = 21
S_df['rv_t'] = S_df['r_t'].rolling(window = window).std().ffill().fillna(0)

# --- Volatility-timed strategy implementation ---

S_df['position'] = S_df.apply(
    lambda row: 1 if row['rv_t'] < 0.8 * sigma else 0, axis = 1
).shift(1).fillna(0)

# --- Strategy Return Computation ---

S_df['strat_r_t'] = S_df['position'] * S_df['r_t']
S_df['Cumulative'] = (1 + S_df['strat_r_t']).cumprod() - 1
S_df['HighValue'] = S_df['Cumulative'].cummax()

# --- Evaluation Metrics ---

annual_return = S_df['strat_r_t'].mean() * n_steps
annual_volatility = S_df['strat_r_t'].std() * np.sqrt(n_steps)
sharpe_ratio = (S_df['strat_r_t'].mean() * np.sqrt(n_steps)) / S_df['strat_r_t'].std()
max_drawdown = min((((S_df['Cumulative'] - S_df['HighValue']) / S_df['HighValue'])).fillna(0))

print(f"Annual Returns: {annual_return:.4f}")
print(f"Annual Volatility: {annual_volatility:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Maximum Drawdown: {max_drawdown:.4f}")



Annual Returns: 1.9257
Annual Volatility: 1.4168
Sharpe Ratio: 1.3592
Maximum Drawdown: -1.1783
